# 简单的RAG(检索增强生成)系统

## 概述

这段代码实现了用于处理和查询PDF文档的基本检索增强生成(RAG)系统。系统将文档内容编码到一个矢量存储中，然后可以查询该存储以检索相关信息。

## 关键部件

1. PDF处理和文本提取
2. 用于可管理处理的文本分块
3. 矢量存储创建使用[FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)和OpenAI嵌入
4. 用于查询已处理文档的检索器设置
5. RAG系统的评估

## 方法详情

### 文档预处理

1. PDF使用PyPDFLoader加载。
2. 使用RecursiveCharacterTextSplitter将文本分成具有指定块大小和重叠的块。

### 文本清理

一个自定义函数`replace_t_with_space`被应用于清理文本块。这可能解决了PDF中特定的格式问题。

### 矢量存储创建

1. OpenAI嵌入用于创建文本块的矢量表示。
2. 从这些嵌入中创建一个FAISS向量存储，用于高效的相似性搜索。

### Retriever设置

1. 检索器被配置为获取给定查询的前2个最相关的块。

### Encoding Function

encode_pdf函数封装了加载、分块、清理和将PDF编码为矢量存储的整个过程。

## 主要特性

1. 模块化设计:编码过程封装在单个函数中，便于重用。
2. 可配置分块:允许调整块大小和重叠。
3. 高效检索:使用FAISS进行快速相似度搜索。
4. 评估:包括评估RAG系统性能的功能。

## 使用示例

代码包括一个测试问题:“气候变化的主要原因是什么?”，这演示了如何使用检索器从处理过的文档中获取相关上下文。

## 评价

该系统包括一个`evaluate_rag`函数来评估检索器的性能，尽管所使用的具体指标在所提供的代码中没有详细说明。

这种方法的好处

1. 可伸缩性:可以通过分组处理来处理大型文档。
2. 灵活性:易于调整参数，如块大小和检索结果的数量。
3. 效率:利用FAISS在高维空间中进行快速相似度搜索。
4. 与高级NLP集成:使用OpenAI嵌入进行最先进的文本表示。

## 结论

这个简单的RAG系统为构建更复杂的信息检索和问答系统提供了坚实的基础。通过将文档内容编码为可搜索的向量存储，它能够有效地检索响应查询的相关信息。这种方法对于需要快速访问大型文档或文档集合中的特定信息的应用程序特别有用。

### Import libraries and environment variables

In [ ]:
import os
import sys
from dotenv import load_dotenv

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path since we work with notebooks
# Load environment variables from a .env file
load_dotenv()

from rag.helper_functions import *
from rag.evaluation.evalute_rag import *

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Read Docs

In [ ]:
path = "../data/Understanding_Climate_Change.pdf"

### Encode document

In [ ]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

In [ ]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

### Create retriever

In [ ]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

### Test retriever

In [ ]:
test_query = "What is the main cause of climate change?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

### Evaluate results

In [15]:
evaluate_rag(chunks_query_retriever)

Answering the question from the retrieved context...
Answering the question from the retrieved context...
Answering the question from the retrieved context...
Answering the question from the retrieved context...
Answering the question from the retrieved context...


Output()

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Output()

Output()

Output()

Output()



Metrics Summary

  - ✅ Correctness (GEval) (score: 0.9300843143497455, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The actual output is factually correct and expands on the expected output by mentioning temperature, precipitation, and wind patterns., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4, reason: None, error: None)
  - ❌ Contextual Relevancy (score: 0.5, threshold: 1.0, strict: False, evaluation model: gpt-4, reason: The score is 0.50 because the context does not directly explain what climate change refers to, but it does discuss related effects like 'Extreme Weather Events', 'Hurricanes and Typhoons', 'Droughts', and 'Flooding'., error: None)

For test case:

  - input: What does climate change refer to?
  - actual output: Climate change refers to significant, long-term changes in the global climate, including temperature, precipitation, and wind patterns, over an extended period.
  - expected output:

✅ Tests finished! Run "deepeval login" to view evaluation results on the web.